<a href="https://colab.research.google.com/github/seoeunkong/MachineLearning/blob/main/chapter05_%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
#테스트 세트와 훈련 세트 만들기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
#훈련세트와 검증세트 만들기
sub_input, val_input, sub_target, val_target = train_test_split(
     train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
#훈련 세트와 테스트 세트 크기 비교
print(train_input.shape, test_input.shape)

#훈련 세트와 검증 세트 크기 비교
print(sub_input.shape, val_input.shape)

(5197, 3) (1300, 3)
(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [ ]:
#교차검증
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00941849, 0.00727606, 0.0077033 , 0.00740933, 0.00743747]), 'score_time': array([0.00092697, 0.00073242, 0.00073886, 0.00075173, 0.000741  ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
#교차 검증의 최종 점수
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
#훈련 세트를 섞은 후 10-폴드 교차 검증을 수행
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [ ]:
#하이퍼파라미터 튜닝
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001,0.0002,0.0003,0.0004,0.0005]}

In [ ]:
#GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만듦.
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)

In [ ]:

gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
#25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 
#전체 훈련 세트에서 자동으로 다시 모델을 훈련한다.
dt =gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
#그리드 서치로 찾은 최적의 매개변수는 bestparams 속성에 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
# 5번의 교차 검증으로 얻은 점수
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
#넘파이의 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
#복잡한 매개변수 조합
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [ ]:
#그리드 서치
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
#최상의 매개변수 조합
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
#랜덤 서치
from scipy.stats import uniform, randint

In [ ]:
rgen = randint(0,10)
rgen.rvs(10)

array([7, 8, 8, 0, 9, 9, 0, 8, 9, 4])

In [ ]:
#이번엔 1000개를 샘플링해서 각 숫자의 개수를 세어보자.
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([120,  83,  95, 103, 124,  93,  93,  96,  77, 116]))

In [ ]:
# 0 ~ 1 사이에서 10개의 실수를 추출
ugen = uniform(0,1)
ugen.rvs(10)

array([0.70824576, 0.1058401 , 0.49906161, 0.93896336, 0.80292413,
       0.07913607, 0.65862914, 0.99399671, 0.76232166, 0.15484703])

In [ ]:
#매개변수의 딕셔너리를 만들기
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
#샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정한다.
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2cf14bd8d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2cf154b2d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2cf14ec410>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2cf14bd050>},
                   random_state=42)

In [ ]:
#최적의 매개변수 조합을 출력하자.
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
#최고의 교차 검증 점수도 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
#최종으로 테스트 세트의 성능
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
